In [1]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from PIL import Image

# This will open the plot in a new browser tab. Other renderers can be found at https://plotly.com/python/renderers/
pio.renderers.default = 'browser'

/Users/hridaybaghar/opt/miniconda3/envs/ooi_dev_2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hridaybaghar/opt/miniconda3/envs/ooi_dev_2/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/hridaybaghar/opt/miniconda3/envs/ooi_dev_2/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /Users/hridaybaghar/opt/miniconda3/envs/ooi_dev_2/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/hridaybaghar/opt/miniconda3/envs/ooi_dev_2/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/hridaybaghar/opt/miniconda3/envs/ooi_dev_2/lib/python3.10/site-packages/torchvis

## Importing and cleaning data

In [2]:
# import in clustering results
df = pd.read_csv('../data/eighty_cluster_results.csv')

In [3]:
# get approximate likelihood of 
def get_confidence_proportion(conf_level):
    """
    Based on Confidence level return approximate confidence proportion
    """
    if conf_level == 3:
        return 1
    elif conf_level == 2:
        return 0.67
    elif conf_level == 1:
        return 0.33
    else:
        return 0

df['confidence_proportion'] = df.signal_confidence.apply(get_confidence_proportion)

## Heatmap Visualization

In [4]:
def df_to_plotly(df):
    return {'z': df.confidence_proportion.tolist(),
            'x': df.time_start.str.replace(' ', 'T').tolist(),
            'y': df.distance_start.tolist()}

In [5]:
fig = go.Figure(data=go.Heatmap(df_to_plotly(df)))
fig.update_layout(
    title='Fin Whale Vocalizations Heatmap',
    xaxis_title='Time',
    yaxis_title='Distance from shore (m)',
    legend_title='Confidence Level',
    hovermode='closest')

# Utility to view original images

In [6]:
# Loading images of data
class DASImageDataset(Dataset):
    def __init__(self, root_dir, transform=transforms.ToTensor()):
        self.root_dir = root_dir
        self.files = os.listdir(self.root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img_file =(os.path.join(self.root_dir, self.files[index]))
        img = Image.open(img_file)
        
        if self.transform:
            img = self.transform(img)
        
        return img, self.files[index]
    
root_dir = '../data/south_dist_chunked_10000to47495_dx5_fs100_bpf14to35Hz'
    
orig_transform = transforms.Compose([transforms.ToTensor()])
orig_images = DASImageDataset(root_dir=root_dir, transform=orig_transform)

In [22]:
import numpy as np
def view_images(partial_time, num_images=40):
    filenames = sorted([(i,f) for i,f in enumerate(orig_images.files) if partial_time in f], key=lambda x: x[1])
    filenames = filenames[:num_images]
    num_images = 40 if num_images > 40 else num_images
    print(f"Displaying {num_images} images. Last file: {filenames[-1][1]}")
    images = [orig_images[i][0].numpy().squeeze().squeeze() for i,f in filenames]

    filenames = [f.split('.png') for _,f in filenames]
    filenames = [(f.split('_start')[0].split('.')[0], f.split('_start')[1].split('.')[0]) for f,_ in filenames]

    fig = px.imshow(np.array(images), facet_col=0, facet_col_wrap=10, binary_string=True, facet_row_spacing=0.1, facet_col_spacing=0.01, height=1000, width=1500)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)

    for i,(t,d) in enumerate(filenames):
        fig.layout.annotations[i]['text'] = f"{pd.to_datetime(t.replace('T', ' ')).strftime('%b %-d, %H:%M:%S')}<br>Start: {d}m"
    fig.show()
    

In [24]:
view_images('2021-11-02T04:01')

Displaying 40 images. Last file: 2021-11-02T04:01:59.834000000_start89333.125.png
